In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string


In [5]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [9]:
fake.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
true.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [11]:
fake['class']=0
true['class']=1

In [12]:
data = pd.concat([fake,true], axis=0)

In [13]:
data.sample(10)

,title,text,subject,date,class
34,As Republicans aim to ride economy to election...,"KING OF PRUSSIA, Pennsylvania/WASHINGTON (Reu...",politicsNews,"December 22, 2017",1
4889,Trump meets Irish PM at White House ahead of S...,(Reuters) - Irish Prime Minister Enda Kenny pr...,politicsNews,"March 16, 2017",1
2612,WATCH: Colbert Asks Tapper How CNN Staff Feel...,Donald Trump is no fan of CNN. He has repeated...,News,"February 9, 2017",0
12782,BREAKING: “TODAY” SHOW’S BILLY BUSH PUNISHED F...,THESE HOLIER THAN THOU PEOPLE ARE SICKENING! S...,politics,"Oct 9, 2016",0
13298,CLASS ACT: Trump Finds Out NY Man’s TRUMP Sign...,Donald Trump has done so many things that you ...,politics,"Aug 9, 2016",0
17897,ANGRY LEFTIST CAUGHT On VIDEO Stealing Student...,***WARNING***This video will make your blood b...,left-news,"Sep 28, 2017",0
20958,SHARIA LAW GRANTS MUSLIM MAN Permission To Mar...,Are you ready for these pedophiles to move in ...,left-news,"Feb 24, 2016",0
15400,Kremlin warns of mutual damage if Ukraine cuts...,MOSCOW (Reuters) - The Kremlin said on Wednesd...,worldnews,"November 8, 2017",1
19157,JESSE WATTERS Takes On Young Anti-Trump Protes...,,left-news,"Feb 11, 2017",0
10090,Trump predicts 'very massive recession' in U.S.,MILWAUKEE (Reuters) - Republican presidential ...,politicsNews,"April 2, 2016",1


In [14]:
data = data.drop(['title','subject','date'], axis=1)

In [17]:
data.reset_index(inplace=True)

In [18]:
data.drop(['index'],axis=1,inplace=True)

In [21]:
data.sample(5)

,text,class
37596,THE HAGUE (Reuters) - A U.N. tribunal on Wedne...,1
871,Nothing is wrong. Pay no attention to the fire...,0
33196,(Reuters) - The National Wrestling Hall of Fam...,1
38485,DUBAI (Reuters) - At least 30 people were kill...,1
32168,BEIJING (Reuters) - China’s Foreign Ministry o...,1


In [22]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', "", text)
    text = re.sub("\\W"," ",text)
    text = re.sub("https?://\S+|www\.\S+","",text)
    text = re.sub("<.*?>","", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n", "", text)
    text = re.sub("\w*\d\w*", "", text)
    return text

In [23]:
data["text"] = data["text"].apply(clean_text)

In [24]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [25]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [26]:
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

LogisticRegression()

In [27]:
prediction = lr.predict(xv_test)
lr.score(xv_test, ytest)

0.9860133630289533

In [28]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [30]:
joblib.dump(vectorizer, 'vectorizer.jb')
joblib.dump(lr, 'lr_model.jb')

['lr_model.jb']